In [3]:
import os
import json
import cohere
from typing import Annotated, TypedDict
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langgraph import StateGraph, tool, ToolNode, ToolMessage
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import ChatOpenAI
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams
from brave_search import BraveSearch
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import QdrantVectorStore


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yunethirikhin/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
import cohere
from langchain.vectorstores import QdrantVectorStore
from langchain.embeddings.base import Embeddings

# Initialize Cohere client
cohere_api_key = 'your_cohere_api_key_here'
cohere_client = cohere.Client(cohere_api_key)

class CohereEmbeddings(Embeddings):
    def embed_documents(self, texts):
        # Get embeddings for a list of texts
        response = cohere_client.embed(texts=texts, model="embed-english-v2.0", truncate="END")
        return response.embeddings

    def embed_query(self, query):
        # Get embedding for a single query
        response = cohere_client.embed(texts=[query], model="embed-english-v2.0", truncate="END")
        return response.embeddings[0]

# Create retriever function using Cohere embeddings
def create_retriever(collection_name, doc_splits):
    cohere_embeddings = CohereEmbeddings()

    # Create Qdrant VectorStore with Cohere embeddings
    vectorstore = QdrantVectorStore.from_documents(
        doc_splits,
        cohere_embeddings,
        url=qdrant_url,
        api_key=qdrant_key,
        collection_name=collection_name,
    )
    return vectorstore.as_retriever()

# Example usage
collection_name = 'your_collection_name'
doc_splits = ['Document text 1', 'Document text 2', 'Document text 3']  # Replace with actual document splits
retriever = create_retriever(collection_name, doc_splits)
